# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path

import numpy as np
import pandas as pd
import sys

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
#| export
def display_mds(*strings:str # any number of strings with text in markdown format
               ):
    """Utility function to display several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: haha'
           )

**bold** and _italic_

- bullet

- bullet

> Note: haha

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display several DataFrame at once"""
    for df in dfs:
        display(df)

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,0.470465,-0.739909,-0.330182,1.119347,0.499924
1,-1.447855,0.312989,-0.613174,-1.255008,1.602491
2,-0.064123,-1.092779,-0.332820,0.310979,0.269590


,0,1,2,3,4,5,6,7,8,9
0,1.189962,-1.599791,-0.938383,-0.312913,-1.237868,-1.572925,1.662769,0.811599,0.280456,-0.523134
1,1.292469,-0.770533,-0.090206,0.225586,-0.088308,-1.003967,0.216139,-1.424453,-0.297965,-0.690568
2,-0.285400,-0.718618,-1.174404,-0.002296,0.141890,-2.241422,0.683999,-1.280102,-0.729977,0.513669


In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of nb for setting up autoreload, paths, ... """
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`